### open hotel data

In [ ]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
import pandas as pd
import jieba
import time
#Get Jieba  Dictionary 
jieba.set_dictionary('./jieba_dict/dict.txt.big')
df = pd.read_csv('D:\\SW_Dev\\MainProject\\DataClearing\\booking_com_ALL10_25_23_06_1109-copy.csv')
print('\ndata qty= ',len(df))
df.head(5)


In [ ]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
#Import Jieba Module
import jieba.posseg as pseg
# Combine the ROW
df_feature_ori = df[['hotel-city','hotel-name','hotel-star-rank','hotel-review-scores','room-price','hotel-feature-key','hotel-feature-list']]

df_feature =df_feature_ori.copy()

# remove the next Line
df_feature['hotel-feature-list'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
# remove the next Line
df_feature['hotel-name'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
# remove the next Line
df_feature['hotel-feature-key'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 

print('')
df_feature['hotel-feature-list'].head(5)
# For each row, combine all the columns into one column
hotel_data_list = df_feature.apply(lambda x: ','.join(x.astype(str)), axis=1)
# hotel_data_list = hotel_data_list.replace({'\n': ''}, regex=True)
print('\nelement = ', hotel_data_list[1])
hotel_data_list.head(5)


In [ ]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
# 斷詞
start_time = time.time()
import jieba.posseg as pseg
#jieba.enable_paddle()
#jieba.enable_parallel(4) --> this will be going wrong in win10 
seg = []
for i in range(len(hotel_data_list)):
    cut_result = list(pseg.cut(hotel_data_list[i], use_paddle=False))
    each_line_list = []
    for w, p in cut_result:
        print("%s: %s"%(w, p))
        if p in ['n', 'nr','ns','nt','nz','nl','ng', 't', 's','vn']:
            print("%s: %s"%(w, p))
            each_line_list.append(w)
    new_line = ' '.join(each_line_list)
    print(seg)
    seg.append(new_line)
print("--- spend %s seconds ---" % (time.time() - start_time))

### 斷詞測試

#### 詞性說明: https://gist.github.com/luw2007/6016931

### 進行斷字斷詞

In [ ]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
print(len(seg))

In [ ]:
#seg = [ s for s in seg if len(s) > 0] 
seg

### 準備存檔，將斷好的字詞存下來。先刪除之前留下的紀錄檔案(segDone.txt)

In [ ]:
!del segDone.txt

### 斷詞結果存檔

In [ ]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
# 斷詞結果存檔
segSaveFile = '.\segmentHotelDataDone_add_hotel_name_v2_1117.txt'
with open(segSaveFile, 'wb') as saveFile:
    for i in range(len(seg)):
        print('\ni= ', i)
        #words = seg[i][0].encode('utf-8')
        words = seg[i].encode('utf-8')
        print('\nwords= ', words)
        if len(words) > 0:
            saveFile.write(words)
            saveFile.write('\n'.encode())

In [ ]:
!head -10 segDone.txt

In [ ]:
!cat segDone.txt | wc -c

### 載入word2vec 函式庫
### 參考網頁：https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
from gensim.models import word2vec

In [ ]:
model.wv.key_to_index

### 一行一行的從檔案中取出句子：一行一個句子，句子中的字必須預先處理好並以空白隔開

In [ ]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
from gensim.models import word2vec
sentences = word2vec.LineSentence('.\segmentHotelDataDone_add_hotel_name_v3_1111.txt')
for line in sentences:
    print('\nline =', line)

### 訓練模型
#### size: 向量維度 = 300
#### sg: 0(CBOW), 1(Skip-gram)

vector_size (int, optional) – 
Dimensionality of the word vectors.

window (int, optional) – 
Maximum distance between the current and predicted word within a sentence.

min_count (int, optional) – 
Ignores all words with total frequency lower than this.

workers (int, optional) – 
Use these many worker threads to train the model (=faster training with multicore machines).

sg ({0, 1}, optional) – 
Training algorithm: 1 for skip-gram; otherwise CBOW.

In [ ]:
start_time = time.time()
# model = word2vec.Word2Vec(sentences, vector_size=150, sg=1, window=10, workers=3, min_count=1)
model = word2vec.Word2Vec(sentences, vector_size=300, sg=1, window=15, workers=3, min_count=1)

print("--- spend %s seconds ---" % (time.time() - start_time))

In [ ]:
model

### 儲存訓練好的word2vec 模型

In [ ]:
# from gensim.models import word2vec
import gensim
# model.save('.\hotel_data_word2vec_v4_1117.model')
# Load Word2Vec
model = gensim.models.Word2Vec.load('.\hotel_data_word2vec_v4_1117.model')

In [ ]:
model.corpus_total_words

In [ ]:
SimiliarHotel = model.wv.most_similar('森浩文旅')
SimiliarHotel

In [ ]:
SimiliarHotel = model.wv.most_similar('好得文旅')
SimiliarHotel

In [ ]:
SimiliarHotel = model.wv.most_similar('西悠飯店台北店')
SimiliarHotel

In [ ]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
import pandas as pd
import jieba
import time
#Get Jieba  Dictionary 
jieba.set_dictionary('./jieba_dict/dict.txt.big')
df = pd.read_csv('D:\\SW_Dev\\MainProject\\DataClearing\\booking_com_ALL10_25_23_06_1109-copy.csv')
print('\ndata qty= ',len(df))
df.head(5)
# Combine the ROW
df_feature_ori = df[['hotel_id','hotel_name']]

df_feature =df_feature_ori.copy()

# remove the next Line
df_feature['hotel_name'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
df_feature.head(10)

recommand = pd.DataFrame()

recommand['hotel_id'] = df_feature['hotel_id']
recommand['hotel_name'] = df_feature['hotel_name']
# recommand_hotel['sim'] = df_feature['hotel_id']

recommand['1st_sim_id'] = df_feature['hotel_id']
recommand['1st_sim_name'] = df_feature['hotel_name']
# recommand_hotel['1st_sim'] = df_feature['hotel_id']

recommand['2nd_sim_id'] = df_feature['hotel_id']
recommand['2nd_sim_name'] = df_feature['hotel_name']
# recommand_hotel['2nd_sim'] = df_feature['hotel_id']

recommand['3rd_sim_id'] = df_feature['hotel_id']
recommand['3rd_sim_name'] = df_feature['hotel_name']
# recommand_hotel['3rd_sim'] = df_feature['hotel_id']

recommand_hotel = recommand.copy()

In [ ]:
# recommand_hotel = pd.DataFrame(columns=['hotel_name', 'hotel_id', '1st_similiar', '1st_similiar_id', '2nd_similiar', '2nd_similiar_id', '3rd_similiar', '3rd_similiar_id'])

In [ ]:
for i in df_feature.index : 
    print('\n hotel-id =', df_feature['hotel_id'][i],' hotel-name = ',df_feature['hotel_name'][i])

In [ ]:
for i in df_feature.index : 
    print(df_feature['hotel_id'][i], df_feature['hotel_name'][i])

# 詞相似度

In [ ]:
SimiliarHotel = model.wv.most_similar('森浩文旅')
SimiliarHotel

In [ ]:
w2v_key_list = model.wv.index_to_key

In [ ]:

model.wv.key_to_index

In [ ]:
df_feature.info()
recommand_hotel.info()            


In [ ]:

HotelList =[]
SimiliarList = []
TargetHotel ='台北109青旅'
SimiliarHotel = model.wv.most_similar(TargetHotel)
w2v_key_list = model.wv.index_to_key
print('\n Word to Vector tpye= ',type(w2v_key_list))
print('\n type', type(SimiliarHotel))
print('\n len', len(SimiliarHotel))
# for i in SimiliarHotel:
#     print('\n Similiar =',i)
#     print('\n i-0 =', i[0])
#     print('\n i-1 =', i[1])
countMAX = 0
for idx in range(0,len(df_feature.index)):
    if
    Target = df_feature['hotel_name'][idx]
    for item in w2v_key_list:

    print('\n target = ',Target )
    SimiliarHotel = model.wv.most_similar(Target)
    for item in SimiliarHotel:
        if TargetHotel in df_feature['hotel_name'][idx]:
            print('\n start = ',  df_feature['hotel_name'][idx])
            print('\n start = ',  df_feature['hotel_name'][idx])
            SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx]]]) 
        if item[0] in df_feature['hotel_name'][idx]:
            print('\n col data = ',idx, df_feature['hotel_name'][idx])                
            if countMAX == 0:                
                print('\ncountMAX =', countMAX)   
                SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx],item[1]]]) 
            if countMAX == 1: 
                print('\ncountMAX =', countMAX)
                SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx],item[1]]]) 
            if countMAX == 2:
                print('\ncountMAX =', countMAX)
                SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx],item[1]]]) 
        countMAX +=1
        if(countMAX > 2):
            countMAX= 0
            break            
                
#                 print('\n Similiar =',i)
#                 print('\n i-0 =', idx[0])
#                 print('\n i-1 =', idx[1])


# for idy in range(0, len(HotelList)):     
#     print('\n idy = ', idy,' HotelList= ',HotelList[idy])

    
# for idy in range(0, len(SimiliarList)):     
#     print('\n idy = ', idy,' SimiliarList= ',SimiliarList[idy])


In [ ]:
# # this is backup Don't Run
# HotelList =[]
# SimiliarList = []
# TargetHotel ='台北109青旅'
# SimiliarHotel = model.wv.most_similar(TargetHotel)
# print('\n type', type(SimiliarHotel))
# print('\n len', len(SimiliarHotel))
# # for i in SimiliarHotel:
# #     print('\n Similiar =',i)
# #     print('\n i-0 =', i[0])
# #     print('\n i-1 =', i[1])
# countMAX = 0
# for item in SimiliarHotel:
#     print('\n item ', item[0],item[1])
#     for idx in range(0,len(df_feature.index)):
#         #     print('\n idx= ', idx)
# #         print('\n hotel in  ', df_feature['hotel_name'][idx])
#         if TargetHotel in df_feature['hotel_name'][idx]:
#                 print('\n start = ',  df_feature['hotel_name'][idx])
#                 print('\n start = ',  df_feature['hotel_name'][idx])
#                 SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx]]]) 

# #             recommand_hotel.at[idx,'hotel_name'] = TargetHotel
# #             recommand_hotel.at[idx,'hotel_id'] = df_feature['hotel_id'][idx]
# #             print('\n id= ', df_feature['hotel_id'][idx])
# #             print('\n name= ', df_feature['hotel_name'][idx])
#         if item[0] in df_feature['hotel_name'][idx]:
#             print('\n col data = ',idx, df_feature['hotel_name'][idx])                
#             if countMAX == 0:                
#                 print('\ncountMAX =', countMAX)   
#                 SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx],item[1]]]) 
                
                
# #                 recommand_hotel.at[idx,'1st_sim_id'] =  df_feature['hotel_id'][idx]
# #                 recommand_hotel.at[idx,'1st_sim_name'] = [df_feature['hotel_name'][idx],item[1]]
# #                 print('\n id= ', recommand_hotel['1st_sim_id'][idx] )
# #                 print('\n name= ', recommand_hotel['1st_sim_name'][idx] )
# #                 recommand_hotel['1st_sim_id'][idx] =  df_feature['hotel_id'][idx]
# #                 recommand_hotel['1st_sim_name'][idx] = [df_feature['hotel_name'][idx],item[1]]
#             if countMAX == 1: 
#                 print('\ncountMAX =', countMAX)
# #                 print('\n ',df_feature['hotel_id'][idx])
# #                 print('\n ',[df_feature['hotel_name'][idx],item[1]])
#                 SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx],item[1]]]) 

# #                 recommand_hotel['2nd_sim_id'][idx] =  df_feature['hotel_id'][idx]
# #                 recommand_hotel['2nd_sim_name'][idx] = [df_feature['hotel_name'][idx],item[1]]
#             if countMAX == 2:
#                 print('\ncountMAX =', countMAX)
# #                 print('\n ',df_feature['hotel_id'][idx])
# #                 print('\n ',[df_feature['hotel_name'][idx],item[1]])
#                 SimiliarList.append([[df_feature['hotel_id'][idx]],[df_feature['hotel_name'][idx],item[1]]]) 

# #                 recommand_hotel['3rd_sim_id'][idx] =  df_feature['hotel_id'][idx]
# #                 recommand_hotel['3rd_sim_name'][idx] = [df_feature['hotel_name'][idx],item[1]]
                    
#     countMAX +=1
#     if(countMAX > 2):
#         countMAX= 0
#         break            

# SimiliarList                
                
# #                 print('\n Similiar =',i)
# #                 print('\n i-0 =', idx[0])
# #                 print('\n i-1 =', idx[1])


# # for idy in range(0, len(HotelList)):     
# #     print('\n idy = ', idy,' HotelList= ',HotelList[idy])

    
# # for idy in range(0, len(SimiliarList)):     
# #     print('\n idy = ', idy,' SimiliarList= ',SimiliarList[idy])


In [ ]:
recommand_hotel.head(3)

In [ ]:
model.wv.similar_by_word('普樂室行旅')

In [ ]:
model.wv.similarity('阿樹國際旅店','美芙精品旅館')

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

In [ ]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


In [ ]:
# print(x_vals)
# print(y_vals)
# print(labels)

x_vals, y_vals, labels = reduce_dimensions(model)

# 模型視覺化

In [ ]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)